# Accessing SENTINEL SAR data

In [106]:
# Import packages
import ee
import geemap
import matplotlib.pyplot as plt
import folium
# import numpy as np
# from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
#%matplotlib inline

In [107]:
# ee.Authenticate()

In [108]:
# Initialize ee
ee.Initialize()

In [109]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [110]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -5.61935403584755,
              36.24020029552919
            ],
            [
              -5.61935403584755,
              35.81216768623635
            ],
            [
              -5.078271376291241,
              35.81216768623635
            ],
            [
              -5.078271376291241,
              36.24020029552919
            ],
            [
              -5.61935403584755,
              36.24020029552919
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [111]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


In [112]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))

In [113]:
ffa_db.bandNames().getInfo()

['VV', 'VH', 'angle']

Area of interest (example):

In [114]:
url = ffa_db.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [115]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(ffa_db.select('VV'),
                   ffa_db.select('VH'),
                   ffa_db.select('VV').divide(ffa_db.select('VH')))

# Create the map object.
m = folium.Map(location=location, zoom_start=11)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)


Vessel area detail:

In [116]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -5.334783200789644,
              36.15078648007727
            ],
            [
              -5.334783200789644,
              36.038728410808304
            ],
            [
              -5.252151077654162,
              36.038728410808304
            ],
            [
              -5.252151077654162,
              36.15078648007727
            ],
            [
              -5.334783200789644,
              36.15078648007727
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

ffa_db2 = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))

url = ffa_db2.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

# Evaluating image time for AIS matching

In [117]:
date = ee.Date(ffa_db.get('system:time_start'))
date_dict = date.getInfo()
print(date_dict)

{'type': 'Date', 'value': 1596478695000}


In [118]:
date = ee.Date(ffa_db.get('system:time_start')).format("yyyy-MM-dd")
date_dict = date.getInfo()
print(date_dict)

2020-08-03


# Saving data in google drive

In [119]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# gauth = GoogleAuth()           
# drive = GoogleDrive(gauth) 
# # Create local webserver and auto handles authentication.
# gauth.LocalWebserverAuth()

In [120]:
# ffa_db.bandNames().getInfo() # we should exclude angle

In [121]:
# ffa_db_f = ffa_db2.select(['VV', 'VH']) # excluding angle band

# # Saving into main directory of drive
# task = ee.batch.Export.image.toDrive(**{
#     'image': ffa_db_f,
#     'description': 'imageToDriveExample',
#     'folder':'Example_folder',
#     'scale': 10,
# })
# task.start()

# # Evaluating task time
# import time 
# while task.active():
#   print('Polling for task (id: {}).'.format(task.id))
#   time.sleep(5)

In [122]:
# # Saving other type of data manually:

# upload_file_list = ['img_access.ipynb']
# for upload_file in upload_file_list:
# 	gfile = drive.CreateFile({'parents': [{'id': '1rHg2p9dEuZragwm8tUDXCzDFBXlWA898'}]})
# 	# Read file and set it as the content of this instance.
# 	gfile.SetContentFile(upload_file)
# 	gfile.Upload() # Upload the file.

# Saving ee.image as TIFF locally


In [123]:
# # first method losses a lot of resolution (there is file size restrictions)

# out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# filename = os.path.join(out_dir, 'sentinel-1.tif')

In [124]:
# geemap.ee_export_image(
#     ffa_db, filename=filename,scale=50, file_per_band=False
# )

In [125]:
# # This package may improve the download but it does not fullt work
# # pip install git+https://github.com/cordmaur/GEES2Downloader.git@main
# from geeS2downloader import GEES2Downloader

In [126]:
# # create a downloader instance
# downloader = GEES2Downloader()

# # call download method passing image and band (1 band at a time on this version)
# downloader.download(img=rgb, band='vis-red')